In [ ]:
# from pyautocad import Autocad
import svgwrite
from collections import defaultdict
# import comtypes
def lockLayers(acad,lock):
    layers = acad.ActiveDocument.Layers
    for layer in layers:
        layer.Lock=lock

def explode_item(acad, item: str):
    items = acad.iter_objects(item)
    for obj in items:
        try:
            obj.Explode()
        except:
            obj.Delete()
            continue

In [ ]:
acad = Autocad(create_if_not_exists=False)
filename='A22002'
doc = acad.Application.Documents.Open(f"D:\MJZ\Projects\TestCAD\data\CAD\A22\{filename}.dwg",False)
# doc = acad.Application.Documents.Open(r"C:\Users\吕富显\Desktop\simple.dwg", False)
# print(acad.doc.Name)
ellipse_list = []
circle_list = []  # {}
line_list = []
arc_list = []
max_explode_block = 3
max_explode_polyline = 1

In [ ]:
#切换模型空间编辑
acad.doc.ActiveSpace=1

In [ ]:
try:
    lockLayers(acad,False)
except comtypes.COMError as e:
    print(e)

In [ ]:
target='Block'
for obj in acad.iter_objects(target):
    print(obj.Layer)
print(len(list(acad.iter_objects(target))))

In [ ]:

explode_id=0
try:
    while explode_id<max_explode_block and 0<len(list(acad.iter_objects('Block')))<1000:
        explode_item(acad, "Block")
        explode_id+=1
except comtypes.COMError as e:
    print(e)

In [ ]:
explode_id = 0
try:
    while explode_id<max_explode_polyline and len(list(acad.iter_objects('Polyline'))):
        explode_item(acad, "Polyline")
        explode_id+=1
except comtypes.COMError as e:
    print(e)

In [ ]:
for obj in acad.iter_objects("Circle"):
    data = {
        "layer": obj.Layer,
        "cx": obj.Center[0],
        "cy": obj.Center[1],
        "r": obj.Radius,
    }
    circle_list.append(data)


In [ ]:
for obj in acad.iter_objects("Arc"):
    try:
        data = {
            "layer": obj.Layer,
            "sx": obj.StartPoint[0],
            "sy": obj.StartPoint[1],
            "r": obj.Radius,
            'angle':obj.TotalAngle,
            'ex':obj.EndPoint[0],
            'ey':obj.EndPoint[1],
        }
        arc_list.append(data)
    except AttributeError:
        continue


In [ ]:
for obj in acad.iter_objects("Ellipse"):
    data = {
        "layer": obj.Layer,
        "cx": obj.Center[0],
        "cy": obj.Center[1],
        "rx": obj.MajorRadius,
        "ry": obj.MinorRadius,
    }
    ellipse_list.append(data)

In [ ]:
for id, obj in enumerate(acad.iter_objects("Line")):
    try:
        data = {
            "layer": obj.Layer,
            "sx": obj.StartPoint[0],
            "sy": obj.StartPoint[1],
            "ex": obj.EndPoint[0],
            "ey": obj.EndPoint[1],
        }
        line_list.append(data)
    except AttributeError:
        continue

In [ ]:
def filterbyLayer(data_list,filter):
    out_list = []
    for data in data_list:
        if filter in data['layer']:
            out_list.append(data)
    return out_list
filter = '_L1'
line_out = filterbyLayer(line_list,filter)
ellipse_out = filterbyLayer(ellipse_list,filter)
circle_out = filterbyLayer(circle_list,filter)
arc_out = filterbyLayer(arc_list,filter)

In [ ]:
def read_json(path):
    import json
    with open(path,'r') as reader:
        return json.loads(reader.read())
json_list = read_json('/home/jesse/Project/SymPoint/A22002_L1.json')
filename='A22002_L1'
svg_write(filename,json_list['line'],json_list['arc'],json_list['circle'],json_list['ellipse'])

In [ ]:
def svg_write(filename,line_list,arc_list,circle_list,ellipse_list,xrate=170,yrate=-170,xoffset=520,yoffset=335):
    import svgwrite
    from collections import defaultdict
    dwg = svgwrite.Drawing(f"{filename}.svg", viewBox='0 0 140 140')
    svg_list = defaultdict(list)
    for item in line_list:
        if item['sx']/xrate+xoffset==item['ex']/xrate+xoffset and item['sy']/yrate+yoffset==item['ey']/yrate+yoffset:
            continue
        line = dwg.path(
            d=f"M {item['sx']/xrate+xoffset},{item['sy']/yrate+yoffset} L {item['ex']/xrate+xoffset},{item['ey']/yrate+yoffset}",
            fill="none",
            stroke="black",
        )
        line['stroke-width']='0.1'
        svg_list[item["layer"]].append(line)
    for item in circle_list:
        if item['r']/xrate == 0:
            continue
        circle = dwg.circle(
            center=(item["cx"] / xrate+xoffset, item["cy"] / yrate+yoffset),
            r=item["r"] / xrate,
            fill="none",
            stroke="black",
        )
        circle['stroke-width']='0.1'
        svg_list[item["layer"]].append(circle)
    for item in ellipse_list:
        if item['rx']/xrate==0 or item['ry']/yrate==0:
            continue
        ellipse = dwg.ellipse(
            center=(item["cx"] / xrate+xoffset, item["cy"] / yrate+yoffset),
            r=(item["rx"] / xrate, item["ry"] / xrate),
            fill="none",
            stroke="black",
        )
        ellipse['stroke-width']='0.1'
        svg_list[item["layer"]].append(ellipse)
    for item in arc_list:
        if item['sx']/xrate+xoffset==item['ex']/xrate+xoffset and item['sy']/yrate+yoffset==item['ey']/yrate+yoffset:
            continue
        arc = dwg.path(
            d=f"M {item['sx']/xrate+xoffset},{item['sy']/yrate+yoffset} A {item['r']/xrate},{item['r']/xrate} 0 {1 if item['angle']>180 else 0} 0 {item['ex']/xrate+xoffset},{item['ey']/yrate+yoffset}",
            fill="none",
            stroke="black",
        )
        arc['stroke-width']='0.1'
        svg_list[item["layer"]].append(arc)
    for item in svg_list.items():
        g = dwg.g()
        for elem in item[1]:
            g.add(elem)
        dwg.add(g)
    dwg.save()

In [ ]:
svg_write(filename,line_list,arc_list,circle_list,ellipse_list)

In [ ]:
def write_json(filename,line_list,circle_list,arc_list,ellipse_list):
    import json
    json_dict={
        "line":line_list,
        "circle":circle_list,
        "arc":arc_list,
        "ellipse":ellipse_list
    }
    with open(f'{filename}.json','w') as writer:
        writer.write(json.dumps(json_dict,indent=4))

In [ ]:
# write_json(filename,line_list,circle_list,arc_list,ellipse_list)
write_json(filename+'_L1',line_out,circle_out,arc_out,ellipse_out)

In [ ]:
# svg分块，并且删除不在区域内坐标，并且修改坐标始终保持参考系为0，0
def svg_segment(x,y,filepath):
    import xml.etree.ElementTree as ET
    from svgpathtools import parse_path

    tree = ET.parse(filepath)
    root = tree.getroot()

    minx, miny, width, height = [int(float(x)) for x in root.attrib['viewBox'].split(' ')]
    minx = x
    miny = y
    xrange=[minx,minx+width]
    yrange=[miny,miny+height]
    

    line_list=[]
    arc_list=[]
    circle_list=[]
    ellipse_list=[]
    layerId=0
    ns = root.tag[:-3]
    for g in root.iter(ns + 'g'):
        layerId+=1
        for path in g.iter(ns + 'path'):
            try:
                path_repre = parse_path(path.attrib['d'])
                startpoint = path_repre.point(0)
                endpoint = path_repre.point(1)
                path_type = path_repre[0].__class__.__name__
                # print(path_repre)
                if (xrange[0]<=startpoint.real<=xrange[1] and yrange[0]<=startpoint.imag<=yrange[1]) or (xrange[0]<=endpoint.real<=xrange[1] and yrange[0]<=endpoint.imag<=yrange[1]):
                    if path_type=='Line':
                        data={
                            'layer':str(layerId),
                            'sx':startpoint.real-minx,
                            'sy':startpoint.imag-miny,
                            'ex':endpoint.real-minx,
                            'ey':endpoint.imag-miny
                        }
                        line_list.append(data)
                    elif path_type=='Arc':
                        data={
                            'layer':str(layerId),
                            'sx':startpoint.real-minx,
                            'sy':startpoint.imag-miny,
                            'r':path_repre[0].radius.real,
                            'angle':270 if path_repre[0].large_arc else 70,
                            'ex':endpoint.real-minx,
                            'ey':endpoint.imag-miny
                        }
                        arc_list.append(data)
            except Exception as e:
                raise RuntimeError("Parse path failed!")

        for circle in g.iter(ns + 'circle'):
            try:
                cx = float(circle.attrib['cx'])
                cy = float(circle.attrib['cy'])
                r = float(circle.attrib['r'])
                if xrange[0]<=cx<=xrange[1] and yrange[0]<=cy<=yrange[1]:
                    data = {
                        'layer':str(layerId),
                        'cx':cx-minx,
                        'cy':cy-miny,
                        'r':r
                    }
                    circle_list.append(data)
            except Exception as e:
                raise RuntimeError("Parse path failed!")
            
        for ellipse in g.iter(ns + 'ellipse'):
            try:
                cx = float(ellipse.attrib['cx'])
                cy = float(ellipse.attrib['cy'])
                rx = float(ellipse.attrib['rx'])
                ry = float(ellipse.attrib['ry'])
                if xrange[0]<=cx<=xrange[1] and yrange[0]<=cy<=yrange[1]:
                    data = {
                        'layer':str(layerId),
                        'cx':cx-minx,
                        'cy':cy-miny,
                        'rx':rx,
                        'ry':ry
                    }
                    ellipse_list.append(data)
            except Exception as e:
                raise RuntimeError("Parse path failed!")
    return line_list,arc_list,circle_list,ellipse_list

In [ ]:
filepath = 'A22002_L1.svg'
for x in range(0,700,140):
    for y in range(0,700,140):
        outpath = f'/home/jesse/Project/SymPoint/data/seg/A22002_L1/{x}_{y}'
        line_list,arc_list,circle_list,ellipse_list = svg_segment(x,y,filepath)
        svg_write(outpath,line_list,arc_list,circle_list,ellipse_list,1,1,0,0)

In [ ]:
def svg_combin(dir,outpath):
    import svgwrite
    import os
    filelist = os.listdir(dir)
    
    line_list=[]
    arc_list=[]
    circle_list=[]
    ellipse_list=[]

    for file in filelist:
        coord = file.replace('.svg','').split('_')
        print(coord)
        lines,arcs,circles,ellipses = svg_parse(0,0,os.path.join(dir,file))
        lines,arcs,circles,ellipses = move(int(coord[0]),int(coord[1]),lines,arcs,circles,ellipses)
        line_list.extend(lines)
        # arc_list.extend(arcs)
        circle_list.extend(circles)
        ellipse_list.extend(ellipses)
    svg_write(outpath,line_list,arc_list,circle_list,ellipse_list,1,1,0,0)

In [ ]:
svg_combin('/home/jesse/Project/SymPoint/data/seg/A22002_L1','total')

In [ ]:
def move(x,y,line_list,arc_list,circle_list,ellipse_list):
    for line in line_list:
        line['sx']=line['sx']+x
        line['sy']=line['sy']+y
        line['ex']=line['ex']+x
        line['ey']=line['ey']+y
    for arc in arc_list:
        arc['sx']=arc['sx']+x
        arc['sy']=arc['sy']+y
        arc['ex']=arc['ex']+x
        arc['ey']=arc['ey']+y
    for circle in circle_list:
        circle['cx']=circle['cx']+x
        circle['cy']=circle['cy']+y
    for ellipse in ellipse_list:
        ellipse['cx']=ellipse['cx']+x
        ellipse['cy']=ellipse['cy']+y
    return line_list,arc_list,circle_list,ellipse_list

In [ ]:
# svg分块，并且删除不在区域内坐标，并且修改坐标始终保持参考系为0，0
def svg_parse(x,y,filepath):
    import xml.etree.ElementTree as ET
    from svgpathtools import parse_path

    tree = ET.parse(filepath)
    root = tree.getroot()

    minx, miny, width, height = [int(float(x)) for x in root.attrib['viewBox'].split(' ')]
    minx = x
    miny = y
    xrange=[minx,minx+width]
    yrange=[miny,miny+height]
    

    line_list=[]
    arc_list=[]
    circle_list=[]
    ellipse_list=[]
    layerId=0
    ns = root.tag[:-3]
    for g in root.iter(ns + 'g'):
        layerId+=1
        for path in g.iter(ns + 'path'):
            try:
                path_repre = parse_path(path.attrib['d'])
                startpoint = path_repre.point(0)
                endpoint = path_repre.point(1)
                path_type = path_repre[0].__class__.__name__
                if path_type=='Line':
                    data={
                        'layer':str(layerId),
                        'sx':startpoint.real,
                        'sy':startpoint.imag,
                        'ex':endpoint.real,
                        'ey':endpoint.imag
                    }
                    line_list.append(data)
                elif path_type=='Arc':
                    data={
                        'layer':str(layerId),
                        'sx':startpoint.real-minx,
                        'sy':startpoint.imag-miny,
                        'r':path_repre[0].radius.real,
                        'angle':270 if path_repre[0].large_arc else 70,
                        'ex':endpoint.real-minx,
                        'ey':endpoint.imag-miny
                    }
                    arc_list.append(data)
            except Exception as e:
                raise RuntimeError("Parse path failed!")

        for circle in g.iter(ns + 'circle'):
            try:
                cx = float(circle.attrib['cx'])
                cy = float(circle.attrib['cy'])
                r = float(circle.attrib['r'])
                
                data = {
                    'layer':str(layerId),
                    'cx':cx-minx,
                    'cy':cy-miny,
                    'r':r
                }
                circle_list.append(data)
            except Exception as e:
                raise RuntimeError("Parse path failed!")
            
        for ellipse in g.iter(ns + 'ellipse'):
            try:
                cx = float(ellipse.attrib['cx'])
                cy = float(ellipse.attrib['cy'])
                rx = float(ellipse.attrib['rx'])
                ry = float(ellipse.attrib['ry'])
                
                data = {
                    'layer':str(layerId),
                    'cx':cx-minx,
                    'cy':cy-miny,
                    'rx':rx,
                    'ry':ry
                }
                ellipse_list.append(data)
            except Exception as e:
                raise RuntimeError("Parse path failed!")
    return line_list,arc_list,circle_list,ellipse_list

In [1]:
from utils.svg_split import svg_combin,svg_parse,svg_write,move,get_minmax
from utils.fileutils import read_json

# l1 = read_json('A22002_L1.json')
full = read_json('json_temp.json')
# line_list,arc_list,circle_list,ellipse_list = l1['line'],l1['arc'],l1['circle'],l1['ellipse']
# line_list,arc_list,circle_list,ellipse_list = move(520,335,170,-170,line_list,arc_list,circle_list,ellipse_list)
# svg_write('L1.svg',700,700,line_list,arc_list,circle_list,ellipse_list)

# line_list,arc_list,circle_list,ellipse_list = full['LINE'],full['ARC'],full['CIRCLE'],full['ELLIPSE']
# line_list,arc_list,circle_list,ellipse_list = move(520,335,170,-170,line_list,arc_list,circle_list,ellipse_list)
# svg_write('full.svg',700,700,line_list,arc_list,circle_list,ellipse_list)

# filepath = '/home/jesse/Project/SymPoint/dataset/all/test/svg_gt/L1.svg'
# for x in range(0,700,140):
#     for y in range(0,700,140):
#         outpath = f'/home/jesse/Project/SymPoint/dataset/split/test/svg_gt/L1_{x}_{y}.svg'
#         line_list,arc_list,circle_list,ellipse_list = svg_parse(x,y,140,140,filepath)
#         line_list,arc_list,circle_list,ellipse_list = move(-x,-y,1,1,line_list,arc_list,circle_list,ellipse_list)
#         svg_write(outpath,140,140,line_list,arc_list,circle_list,ellipse_list)

In [3]:
from collections import defaultdict
layers = defaultdict(dict)
for type in full.keys():
    for id,item in enumerate(full[type]):
        if type not in layers[item["layer"]].keys():
            layers[item["layer"]][type] = []
        layers[item["layer"]][type].append(item)
# layers

In [4]:
key_list = defaultdict(dict)
for key in layers.keys():
    splits = key.split('$0$')[:-1]
    temp = ''
    for split in splits:
        temp+=split
        if temp not in key_list:
            
            key_list[temp]=defaultdict(dict)
            key_list[temp]['LINE']=[]
            key_list[temp]['ELLIPSE']=[]
            key_list[temp]['ARC']=[]
            key_list[temp]['CIRCLE']=[]
        temp+='$0$'
# print(key_list['A2-03 一层平面图'])

In [ ]:
key_list = defaultdict(dict)
for key in layers.keys():
    splits = key.split('$0$',1)[0]
    # print(splits)
    if splits!='0' and splits not in key_list.keys():
        key_list[splits]=defaultdict(list)
key_list

In [5]:
for target in key_list:
    for key,value in layers.items():
        # print(key,value.keys())
        if key!='0' and target in key:
            for type in value.keys():
                key_list[target][type].extend(value[type])
    

In [ ]:
def write_json(filepath,data):
    import json
    with open(filepath,'w') as writer:
        writer.write(json.dumps(data,indent=4,ensure_ascii=False))
write_json('layer.json',key_list)

In [ ]:
get_minmax(line_list,arc_list,circle_list,ellipse_list)

In [12]:
data = read_json('layer.json')
targetHW=700
for key,value in data.items():
    filename=f'{key}.svg'
    line_list=value['LINE']
    arc_list=value['ARC']
    circle_list=value['CIRCLE']
    ellipse_list=value['ELLIPSE']
    xmin,xmax,ymin,ymax = get_minmax(line_list,arc_list,circle_list,ellipse_list)
    rate = max((xmax-xmin),(ymax-ymin))/targetHW
    if int(rate)==0:
        continue
    mx=-1*xmin/rate
    my=ymax/rate
    dx=rate
    dy=-1*rate if ymin<0 else rate
    line_list,arc_list,circle_list,ellipse_list = move(mx,my,rate,-1*rate,line_list,arc_list,circle_list,ellipse_list)
    svg_write(filename,targetHW,targetHW,line_list,arc_list,circle_list,ellipse_list)

In [1]:
min(1,2)

1